In [1]:
from subtitles import get_es_and_fr_subtitles_from_files
subtitles = get_es_and_fr_subtitles_from_files(1)

./Movies/Aquí No Hay Quien Viva 1/A1_t00_es.srt ./Movies/Aquí No Hay Quien Viva 1/A1_t00_fr.srt
in load_subtitles file :  <_io.TextIOWrapper name='./Movies/Aquí No Hay Quien Viva 1/A1_t00_es.srt' mode='r' encoding='utf-8'>
in load_subtitles file :  <_io.TextIOWrapper name='./Movies/Aquí No Hay Quien Viva 1/A1_t00_fr.srt' mode='r' encoding='utf-8'>


In [2]:
print(subtitles)

[{'id': '1', 'timestamp': '00:00:00,000 --> 00:00:00,956', 'spanish': 'Subtítulos: Atresmedia', 'french': 'Sous-titres : Atresmedia'}, {'id': '2', 'timestamp': '00:00:26,080 --> 00:00:29,994', 'spanish': '"Aquí, aquí no hay quien viva,\naquí no, aquí no."', 'french': '"Ici, personne ne peut vivre ici,\npas ici, pas ici."'}, {'id': '3', 'timestamp': '00:00:38,720 --> 00:00:42,839', 'spanish': '-¡Vicenta! Ven que esto empieza ya.\n-No, voy a acostarme.', 'french': '-Vicenta ! Allez, ça commence maintenant.\n-Non, je vais me coucher.'}, {'id': '4', 'timestamp': '00:00:42,920 --> 00:00:46,879', 'spanish': 'Tengo unos ardores..\n-Normal, soltera toda tu vida.', 'french': "J'ai des ardeurs...\n-Normal, tu es célibataire depuis toujours."}, {'id': '5', 'timestamp': '00:00:46,960 --> 00:00:48,758', 'spanish': '-Ardor de estómago.', 'french': "-Brûlures d'estomac."}, {'id': '6', 'timestamp': '00:00:49,560 --> 00:00:53,269', 'spanish': '¿Por qué siempre me recuerdas\nque no me he casado? Yo deci

In [3]:
from database import Media, Subtitle, engine


In [4]:
import datetime
from sqlalchemy.orm import Session
from datetime import time
def parse_srt_timestamp(srt_time: str) -> time:
    return datetime.strptime(srt_time, "%H:%M:%S,%f").time()

def parse_srt_time_range(srt_time_range: str) -> tuple[time, time]:
    """Convert one timestamp line of  SRT in two objects datetime.time (start_time, end_time)."""
    start_str, end_str = srt_time_range.split(" --> ")
    start_time = datetime.datetime.strptime(start_str, "%H:%M:%S,%f").time()
    end_time = datetime.datetime.strptime(end_str, "%H:%M:%S,%f").time()
    return start_time, end_time

with Session(engine) as session:

    serie1 = Media(
        title = "Aquí no hay qien viva",
        media_type = "series",
        season = 1,
        series_number = 1,
        series_title = "Érase una mudanza",
        disc_number = 1
    )
    
    for subtitle in subtitles:
        start_time, end_time = parse_srt_time_range(subtitle['timestamp'])
        entry = Subtitle(
            id = subtitle['id'],
            media = serie1,
            start_time = start_time,
            end_time = end_time,
            spanish_text = subtitle['spanish'],
            french_text = subtitle['french']
        )
        session.add(entry)
    session.commit()
        

In [7]:
from sqlalchemy import select, and_
from sqlalchemy.orm import Session

stmt = select(Subtitle).where(Subtitle.id == 11)
                        
with Session(engine) as session:
    for entry in session.scalars(stmt):
        print(entry.id)
        print(entry.spanish_text)
        print('------')
        print(entry.french_text)

11
-Si le digo a las 11, es a las 11.
¿Por qué esta niña nunca hace caso?
------
-Si je lui dis à 11 heures, c'est à 11 heures.
Pourquoi cette enfant n'écoute-t-elle jamais ?


In [20]:
subtitles[1].start_time

datetime.time(0, 0, 26, 80000)

In [12]:
subtitles[2].spanish_text

'-¡Vicenta! Ven que esto empieza ya.\n-No, voy a acostarme.'

In [13]:
from sqlalchemy import create_engine, MetaData

# Connexion à la base de données
engine = create_engine("sqlite:///subtitles.db")  
metadata = MetaData()
metadata.reflect(bind=engine)  # Récupère la structure actuelle de la base

# Suppression de la table "subtitles" si elle existe
if "subtitles" in metadata.tables:
    metadata.tables["subtitles"].drop(engine)
    print("Table 'subtitles' supprimée avec succès ! 🎉")
else:
    print("La table 'subtitles' n'existe pas.")


Table 'subtitles' supprimée avec succès ! 🎉


In [1]:
from database import update_subtitle
update_subtitle({'id' : 11})

{'database_name': 'sqlite:////Users/pam/assimil/db/assimil_spanish.db', 'basic_audio_directory': '/Users/pam/AssimilAudio/Basic', 'using_spanish_audio_directory': '/Users/pam/AssimilAudio/UsingSpanish', 'html_basic_lessons_directory': '/Users/pam/assimil/lessons/Basic/html', 'html_using_spanish_lessons_directory': '/Users/pam/assimil/lessons/UsingSpanish/html', 'databases_directory': '/Users/pam/assimil/db', 'basic_french_translation_lessons_directoy': '/Users/pam/assimil/lessons/Basic/new', 'using_spanish_french_translation_lessons_directoy': '/Users/pam/assimil/lessons/UsingSpanish/new', 'word_dict_logfilename': '/Users/pam/assimil/word_dict_logfile.log', 'notes_directory': '/Users/pam/assimil/lessons/notes'}
11 -Si le digo a las 11, es a las 11.
¿Por qué esta niña nunca hace caso? -Si je dis 11 heures, il est 11 heures.
Pourquoi cet enfant n'écoute-t-il jamais ?


In [3]:
import os
os.listdir("Movies/Aquí No Hay Quien Viva 1/A1_t00.m4v")

NotADirectoryError: [Errno 20] Not a directory: 'Movies/Aquí No Hay Quien Viva 1/A1_t00.m4v'

In [6]:
import ffmpeg
input_file = "Movies/Aquí No Hay Quien Viva 1/A1_t00.m4v"
output_file = "Movies/audio_extrait.mp3"
start_time = "00:20:33"
duration = 25

try:
    (
        ffmpeg
        .input(input_file, ss=start_time)
        .output(output_file, vn=True, t=duration)
        .run()
    )
    print(f"L'extrait audio a été extrait et sauvegardé dans '{output_file}'")
except ffmpeg.Error as e:
    print(f"Erreur lors de l'exécution de FFmpeg : {e.stderr}")


Erreur lors de l'exécution de FFmpeg : None


ffmpeg version 6.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

In [18]:
import ffmpeg

input_file = 'Movies/Aquí No Hay Quien Viva 1/A1_t00.m4v'
output_file = 'extrait_audio.mp3'
start_time = '00:20:33'
duration = 30  # Durée en secondes

try:
    (
        ffmpeg
        .input(input_file, ss=start_time)
        .output(output_file, t=duration)
        .run()
    )
    print(f"L'extrait audio a été extrait et sauvegardé dans '{output_file}'")
except ffmpeg.Error as e:
    print(f"Erreur lors de l'exécution de FFmpeg : {e.stderr.decode('utf8')}")

ffmpeg version 6.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

L'extrait audio a été extrait et sauvegardé dans 'extrait_audio.mp3'


size=     470kB time=00:00:29.98 bitrate= 128.3kbits/s speed=27.7x    
video:0kB audio:469kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.096743%


In [ ]:
ffmpeg -i "Movies/Aquí No Hay Quien Viva 1/A1_t00.m4v" -ss 00:20:33 -to 00:20:57 -vn "audio_extrait.mp3"
ffmpeg -i votre_video.mp4 -ss 00:01:30 -to 00:02:00 -vn audio_extrait.mp3

In [19]:
from google.cloud import speech_v1 as speech

audio_file = 'extrait_audio.mp3'

def transcribe_audio(audio_file):
    """Transcrit un fichier audio en espagnol."""
    client = speech.SpeechClient()

    with open(audio_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        language_code="es-ES",  # Vous pouvez ajuster la variante régionale
    )

    response = client.recognize(config=config, audio=audio)

    transcript = ""
    for result in response.results:
        transcript += result.alternatives[0].transcript + " "
    return transcript.strip()


texte_transcrit = transcribe_audio(audio_file)
print(f"Texte transcrit : {texte_transcrit}")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.